# ✉️ 消息
  <img src="./assets/LC_Messages.png" width="500">

消息是 LangChain 中模型上下文的基本单位。它们表示模型的输入和输出，携带与 LLM 交互时表示对话状态所需的内容和元数据。

## 设置

加载和/或检查所需的环境变量

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 文件加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env("example.env")

## 人类👨‍💻 和 AI 🤖 消息

In [2]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="openai:gpt-5-nano", 
    system_prompt="You are a full-stack comedian"
)

In [3]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [4]:
print(result["messages"][-1].content)

Hey there! I’m doing well—stack is healthy, coffee levels are optimal, and I’ve got a fresh batch of jokes ready. How are you?

A couple quick techy one-liners to start:
- Why do programmers prefer dark mode? Because light attracts bugs.
- I would tell you a UDP joke, but you might not get it.
- My code runs on coffee and sarcasm—mostly sarcasm.

What can I help you with today? Want a joke, a debugging tip, a quick code snippet, or a plan for a stand-up bit about full-stack life?


In [5]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [6]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I’m doing well—stack is healthy, coffee levels are optimal, and I’ve got a fresh batch of jokes ready. How are you?

A couple quick techy one-liners to start:
- Why do programmers prefer dark mode? Because light attracts bugs.
- I would tell you a UDP joke, but you might not get it.
- My code runs on coffee and sarcasm—mostly sarcasm.

What can I help you with today? Want a joke, a debugging tip, a quick code snippet, or a plan for a stand-up bit about full-stack life?



### 替代格式
#### 字符串
在某些情况下，LangChain 可以从上下文中推断角色，简单的字符串就足以创建消息。

In [ ]:
agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a terse sports poet.",  # 这在底层是一个 SystemMessage
)

In [ ]:
result = agent.invoke({"messages": "Tell me about baseball"})   # 这在底层是一个 HumanMessage
print(result["messages"][-1].content)

#### 字典

In [9]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Sprinters surge forward
Sun sears the track as feet scream
Finish lines shiver


有多种角色：
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## 输出格式
### 消息
让我们创建一个工具，这样智能体将创建一些工具消息。

In [ ]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """检查给定的俳句文本是否恰好有 3 行。

    如果正确则返回 None，否则返回错误消息。
    """
    # 将文本拆分成行，忽略前导/尾随空格
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [11]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [12]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn


In [13]:
result["messages"][-1].content

'Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn'

In [14]:
print(len(result["messages"]))

4


In [15]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_uOLf0ZxcBTNBT2a77IZJi1jO)
 Call ID: call_uOLf0ZxcBTNBT2a77IZJi1jO
  Args:
    text: Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn


### 其他有用信息
上面的打印消息只是选择了消息列表中存储的部分信息。让我们深入了解所有可用的信息！

In [16]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='13b18ae4-f92e-4582-925a-39d7984806a8'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 555, 'prompt_tokens': 170, 'total_tokens': 725, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbI0V3W0uPCDlcnD4GYTydYxCsAJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0055426a-ee06-4daa-a36c-4074fa9c1ceb-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn'}, 'id': 'call_uOLf0ZxcBTNBT2a77IZJi1jO', 'type': 'tool_call'}], usage_

你可以只选择最后一条消息，你可以看到最终消息来自哪里。

In [17]:
result["messages"][-1]

AIMessage(content='Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 231, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--61eb6a33-1bda-46d7-8d00-f3aaea2487a3-0', usage_metadata={'input_tokens': 231, 'output_tokens': 21, 'total_tokens': 252, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
result["messages"][-1].usage_metadata

{'input_tokens': 231,
 'output_tokens': 21,
 'total_tokens': 252,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [19]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 21,
  'prompt_tokens': 231,
  'total_tokens': 252,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### 自己尝试！
更改系统提示，使用 `pretty_printer` 打印一些消息或自己深入研究 `results`。注意人类消息、AI 消息和工具消息以及它们关联的一些元数据。注意最终结果如何提供智能体活动的完整历史！

In [ ]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="在这里填写你的系统提示",
)

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()